# Laboratorio 8

## Alberi bilanciati ctd. - AVL 

### Alberi AVL 
1. Gli alberi AVL sono alberi binari di ricerca auto-bilancianti, dove ogni nodo ha un fattore di bilanciamento che indica la differenza tra l'altezza dei sottoalberi sinistro e destro.

2. Se il fattore di bilanciamento di un nodo è maggiore di 1 o minore di -1, l'albero deve essere ri-bilanciato per mantenere la proprietà di auto-bilanciamento.

3. Ci sono quattro tipi di rotazioni che possono essere eseguite per ri-bilanciare un albero AVL: rotazione a destra, rotazione a sinistra, rotazione doppia a destra e rotazione doppia a sinistra.

4. Durante l'inserimento o l'eliminazione di un nodo, la struttura dell'albero AVL può essere modificata, quindi l'albero deve essere ri-bilanciato per mantenere la proprietà di auto-bilanciamento.

5. L'altezza di un albero AVL è sempre logaritmica rispetto al numero di nodi, il che significa che le operazioni di ricerca, inserimento ed eliminazione richiedono un tempo $O(logn)$, dove n è il numero di nodi nell'albero.

In [2]:
class AVLTree:
    """Implementazione di un albero AVL.

    Attributes:
        root (Node): La radice dell'albero AVL.

    """

    class Node:
        """Rappresentazione di un nodo dell'albero AVL.

        Gli oggetti Node rappresentano un nodo dell'albero AVL che contiene un valore e due figli,
        sinistro e destro.

        Attributes:
            value (int): Il valore del nodo.
            left (Node): Il sotto-albero sinistro del nodo.
            right (Node): Il sotto-albero destro del nodo.
            height (int): L'altezza del sotto-albero radicato in questo nodo.
        """

        def __init__(self, value):
            self.value = value
            self.left = None
            self.right = None
            self.height = 1

    def __init__(self):
        """Inizializza un nuovo albero AVL."""
        self.root = None

    def insert(self, value):
        """Inserisce un nuovo valore nell'albero AVL.

        Args:
            value (int): Il valore da inserire nell'albero.

        """
        self.root = self._insert_helper(self.root, value)

    def delete(self, value):
        """Elimina un valore dall'albero AVL, se presente.

        Args:
            value (int): Il valore da eliminare dall'albero.

        """
        self.root = self._delete_helper(self.root, value)

    def search(self, value):
        """Cerca un valore nell'albero AVL e restituisce True se presente, False altrimenti.

        Args:
            value (int): Il valore da cercare nell'albero.

        Returns:
            bool: True se il valore è presente nell'albero, False altrimenti.
        """
        return self._search_helper(self.root, value)

    def _insert_helper(self, node: Node, value):
        """Helper method per l'inserimento di un valore nell'albero AVL."""
        if node is None:
            return AVLTree.Node(value)

        if value < node.value:
            node.left = self._insert_helper(node.left, value)
        else:
            node.right = self._insert_helper(node.right, value)

        node.height = 1 + max(self._get_height(node.left),
                              self._get_height(node.right))

        balance = self._get_balance(node)

        #################################################
        # Balance >  1 : sottoalbero sx è troppo profondo
        # Balance < -1 : sottoalbero dx è troppo profondo
        #################################################

        # Caso SS
        if balance > 1 and value < node.left.value:
            return self._rotate_right(node)

        # Caso DD
        if balance < -1 and value > node.right.value:
            return self._rotate_left(node)

        # Caso DS
        if balance > 1 and value > node.left.value:
            node.left = self._rotate_left(node.left)
            return self._rotate_right(node)

        # Caso SD
        if balance < -1 and value < node.right.value:
            node.right = self._rotate_right(node.right)
            return self._rotate_left(node)

        return node

    def _delete_helper(self, node: Node, value):
        """Helper method per l'eliminazione di un valore dall'albero AVL."""
        if node is None:
            return node

        # Attraverso l'albero per cercare il nodo da eliminare
        if value < node.value:
            node.left = self._delete_helper(node.left, value)
        elif value > node.value:
            node.right = self._delete_helper(node.right, value)
        # Una volta trovato, agisco
        else:
            # Come per gli alberi binari di ricerca, osservo se manca uno dei figli
            if node.left is None:
                temp = node.right
                node = None
                return temp
            elif node.right is None:
                temp = node.left
                node = None
                return temp

            # Se così non è, allora cerco il nodo con
            # valore minimo nel sottoalbero destro,
            # lo sostituisco con il nodo corrente, e ripeto
            # il processo di eliminazione nel sottoalbero dx
            temp = self._get_min_value_node(node.right)
            node.value = temp.value
            node.right = self._delete_helper(node.right, temp.value)

        if node is None:
            return node

        node.height = 1 + max(self._get_height(node.left),
                              self._get_height(node.right))

        # Calcoliamo la differenza tra le altezze dei figli
        balance = self._get_balance(node)

        # Ed aggiustiamo il bilanciamento dell'albero nel caso
        # la rimozione del nodo precedente abbia invalidato la proprietà dell'AVL

        # Reminder ;)
        #################################################
        # Balance >  1 : sottoalbero sx è troppo profondo
        # Balance < -1 : sottoalbero dx è troppo profondo
        #################################################

        # Caso SS per la rimozione
        #          z                                      y
        #         / \                                   /   \
        #        y   T4      Right Rotate (z)          x      z
        #       / \          - - - - - - - - ->      /  \    /  \
        #      x   T3                               T1  T2  T3  T4
        #     / \
        #   T1   T2

        if balance > 1 and self._get_balance(node.left) >= 0:
            return self._rotate_right(node)

        # Caso DD per la rimozione
        #    z                                y
        #  /   \                            /   \ 
        # T1    y     Left Rotate(z)       z      x
        #     /  \   - - - - - - - ->     / \    / \
        #    T2   x                      T1  T2 T3  T4
        #    / \
        #   T3  T4
        if balance < -1 and self._get_balance(node.right) <= 0:
            return self._rotate_left(node)

        # Caso SD per la rimozione
        #      z                               z                           x
        #     / \                            /   \                        /  \ 
        #    y   T4  Left Rotate (y)        x    T4  Right Rotate(z)    y      z
        #   / \      - - - - - - - - ->    /  \      - - - - - - - ->  / \    / \
        #  T1  x                          y    T3                    T1  T2 T3  T4
        #     / \                        / \
        #   T2   T3                    T1   T2
        if balance > 1 and self._get_balance(node.left) < 0:
            node.left = self._rotate_left(node.left)
            return self._rotate_right(node)

        # Caso DS per la rimozione
        #   z                            z                            x
        #  / \                          / \                          /  \ 
        # T1   y   Right Rotate (y)    T1   x      Left Rotate(z)   z      y
        #     / \  - - - - - - - - ->     /  \   - - - - - - - ->  / \    / \
        #    x   T4                      T2   y                  T1  T2  T3  T4
        #   / \                              /  \
        # T2   T3                           T3   T4
        if balance < -1 and self._get_balance(node.right) > 0:
            node.right = self._rotate_right(node.right)
            return self._rotate_left(node)

        return node

    def _rotate_left(self, node: Node):
        """Esegue una rotazione a sinistra di un nodo dell'albero AVL."""
        y = node.right
        T2 = y.left

        y.left = node
        node.right = T2

        node.height = 1 + max(self._get_height(node.left),
                              self._get_height(node.right))
        y.height = 1 + max(self._get_height(y.left),
                            self._get_height(y.right))

        return y

    def _rotate_right(self, node: Node):
        """Esegue una rotazione a destra di un nodo dell'albero AVL."""
        y = node.left
        T3 = y.right

        y.right = node
        node.left = T3

        node.height = 1 + max(self._get_height(node.left),
                              self._get_height(node.right))
        y.height = 1 + max(self._get_height(y.left),
                            self._get_height(y.right))

        return y

    def _get_height(self, node: Node):
        """Restituisce l'altezza di un nodo."""
        if node is None:
            return 0
        return node.height

    def _get_balance(self, node: Node):
        """Restituisce il fattore di bilanciamento di un nodo."""
        if node is None:
            return 0
        return self._get_height(node.left) - self._get_height(node.right)

    def _get_min_value_node(self, node: Node):
        """Restituisce il nodo con il valore minimo nell'albero."""
        if node is None or node.left is None:
            return node
        return self._get_min_value_node(node.left)

    def _search_helper(self, node: Node, value):
        """Helper method per la ricerca di un valore nell'albero AVL."""
        if node is None:
            return False

        if node.value == value:
            return True

        if value < node.value:
            return self._search_helper(node.left, value)
        else:
            return self._search_helper(node.right, value)
        
def nice_tree_print(node, space=0, offset=5):
    if node is None:
        return

    space += offset

    nice_tree_print(node.right, space)

    print(end=' ' * (space - offset))
    print(node.value)

    nice_tree_print(node.left, space)

In [22]:
tree = AVLTree()
tree.insert(5)
tree.insert(3)
tree.insert(7)
tree.insert(2)
tree.insert(4)
tree.insert(6)
tree.insert(8)

nice_tree_print(tree.root)

          8
     7
          6
5
          4
     3
          2


In [23]:
tree.insert(1)
nice_tree_print(tree.root)

          8
     7
          6
5
          4
     3
          2
               1


In [24]:
# Sbilanciamento in arrivo, caso SS e conseguente rotazione a dx rispetto a 2
tree.insert(0)
nice_tree_print(tree.root)

          8
     7
          6
5
          4
     3
               2
          1
               0


In [21]:
tree.delete(5)
nice_tree_print(tree.root)

          8
     7
6
          4
     3
               2
          1
               0


### Red Black Trees

1. I RBT sono una tipologia di albero binario di ricerca auto-bilanciante, utilizzati per implementare strutture dati come mappe, insiemi e tabelle hash. 
2. In un RBT, ogni nodo è colorato di rosso o di nero, e viene applicato un insieme di regole per assicurarsi che l'albero rimanga bilanciato. Queste regole includono il fatto che ogni nodo deve essere o nero o rosso, che la radice deve essere nera, che ogni foglia (cioè i nodi nulli) deve essere nera, e che ogni nodo rosso deve avere due figli neri.

## Analisi ammortizzata

Supponiamo di eseguire una sequenza di `n` operazioni su una struttura dati in cui l'operazione `i`-esima costa `i` se `i` è una potenza esatta di 2, e `1` altrimenti. 
1. Utilizzare l'analisi aggregata per determinare il costo ammortizzato per operazione.
2. Utilizzare il metodo del banchiere per determinare il costo ammortizzato per operazione.
3. Utilizzare il metodo del potenziale per determinare il costo ammortizzato per operazione.

**Soluzioni:**
1. Sia $n$ arbitrario e sia $c(i)$ il costo dell'$i$-esima operazione. Allora abbiamo:
    $$
    \sum_{i=1}^{n} c(i) = \sum_{i=1}^{\lceil\log n\rceil} 2^i + \sum_{i=1}^{n \text{ non potenza di 2}} 1 \\
    \qquad \leq \sum_{i=1}^{\lceil\log n\rceil} 2^i + n \\
    \qquad = 2^{\lceil\log n\rceil+1} - 1 + n \\
    \qquad \leq 2^{\lceil\log n\rceil+1} + n \\
    \qquad \leq 2n + n \\
    \qquad \leq 3n \in O(n).
    $$
    Per trovare la media, dividiamo per $n$ e otteniamo che il costo ammortizzato per operazione è $O(1)$.

2. Sia $c_i$ il costo dell'$i$-esima operazione, uguale a $i$ se una potenza di $2$ altrimenti 1, e sia $credit$ il credito accumulato. Fissiamo il costo ammortizzato di ogni operazione a $3$.
   Se $i$ non è una potenza esatta di $2$, allora il costo effettivo dell'operazione è $1$ e il credito accumulato è aumentato di $2$.
   Se $i$ è una potenza esatta di $2$, allora il costo effettivo dell'operazione è $c_i$ e, se $credit \geq c_i$, il credito viene utilizzato e ridotto di $c_i$. In caso contrario, il credito viene utilizzato per pagare parte del costo effettivo dell'operazione, e viene poi ridotto a $0$.
   Di seguito, riportiamo una tabella che mostra il costo effettivo di ogni operazione, il costo ammortizzato e il credito accumulato dopo ogni operazione:

    | Operazione | Costo effettivo | Costo ammortizzato | Credito accumulato |
    |------------|----------------|--------------------|---------------------|
    |     1      |       1        |          3         |           2           |
    |     2      |       2        |          3         |           3           |
    |     3      |       1        |          3         |           5           |
    |     4      |       4        |          3         |           4           |
    |     5      |       1        |          3         |           6           |
    |     6      |       1        |          3         |           8           |
    |     7      |       1        |          3         |          10          |
    |     8      |       8        |          3         |           5           |
    |     9      |       1        |          3         |           7           |
    |     10     |       1        |          3         |           9           |

    Poiché il costo ammortizzato di ogni operazione è $3$, la somma dei costi ammortizzati di tutte le operazioni è $3n$.

    Inoltre, sappiamo dal punto precedente che $\sum_{i=1}^{n} c_i < 3n$. Quindi abbiamo:

    $$\sum_{i=1}^{n} \hat{c}_i \geq \sum_{i=1}^{n} c_i \quad \Rightarrow \quad credit = costo_{ammortizzato} - costo_{effettivo} \geq 0.$$

    Poiché il costo ammortizzato di ogni operazione è $O(1)$ e la quantità di credito non diventa mai negativa, il costo totale di $n$ operazioni è $O(n)$.

3. La funzione potenziale è definita come segue:
   - $\Phi(D_0) = 0$, e $\Phi(D_i) = 2i-2^{1+\lfloor\log i\rfloor}$ per $i>0$.

   Per l'operazione 1, abbiamo:
   - $\hat{c}_1 = c_1 + \Phi(D_1) - \Phi(D_0) = 1+2i-{2^{1+\lfloor\log 1\rfloor}} - 0 = 1$.

   Per l'operazione $i>1$, se $i$ non è una potenza di 2, allora:
   - $\hat{c}_i = c_i + \Phi(D_i) - \Phi(D_{i-1}) = 1+2i-2^{1+\lfloor\log i\rfloor} - (2(i-1)-2^{1+\lfloor\log(i-1)\rfloor}) = 3$.

   Se invece $i$ è una potenza di 2, allora:
   - $\hat{c}_i = c_i + \Phi(D_i) - \Phi(D_{i-1}) = i+2i-2^{1+j} - (2(i-1)-2^{1+j-1}) = i + 2i - 2i -2i +2 +i = 2$.

   dove $i = 2^j$.

   In entrambi i casi abbiamo $\hat{c}_i \leq 3$, quindi il costo totale di $n$ operazioni è $O(n)$.